In [1]:
import pandas as pd
from py2neo import Graph, Node, Relationship, NodeMatcher

In [6]:
# 读取csv文件
group_df = pd.read_csv(r'./csv/group.csv')
team_df = pd.read_csv(r'./csv/team.csv')

In [7]:
group_df

,group
0,A
1,B
2,C
3,D
4,E
5,F
6,G
7,H


In [3]:
team_df

,group,team,seed
0,A,老司机名人堂,1
1,A,通来球会,0
2,A,南葛,0
3,A,0773桂林联队,0
4,B,红魔新青年,1
5,B,深圳原点海归,0
6,B,热博体育,0
7,B,广外校友队,0
8,C,五山旅店,1
9,C,华东交大校友队,0


In [8]:
# 连接Neo4j服务
graph = Graph("http://localhost:7474",auth=("neo4j","Bengbu_2001"))

In [10]:
# 创建分组节点
for i in range(group_df.shape[0]):
    name = group_df.iloc[i]['group']
    node = Node("Group", 
                name=name
                )
    graph.create(node)

In [14]:
# 创建Team节点
for i in range(team_df.shape[0]):
    team = team_df.iloc[i]['team']
    is_seed = int(team_df.iloc[i]['seed'])
    node = Node("Team", name=team, seed=is_seed)
    graph.create(node)

In [15]:
# 创建分组——Team关系
matcher = NodeMatcher(graph)
for i in range(team_df.shape[0]):
    group = team_df.iloc[i]['group']
    group_node = matcher.match("Group", name=group).first()
    
    team = team_df.iloc[i]['team']
    team_node = matcher.match("Team", name=team).first()
    
    relationship = Relationship(team_node, 'TEAM_IN', group_node)
    graph.create(relationship)

In [16]:
# 读取比赛结果
result_df = pd.read_csv(r'./csv/result.csv')

In [17]:
result_df

,team1,team2,result
0,老司机名人堂,通来球会,平
1,南葛,0773桂林联队,负
2,红魔新青年,深圳原点海归,胜
3,热博体育,广外校友队,胜
4,五山旅店,华东交大校友队,胜
5,翠柏欢乐时光,电子科大中山学院校友队,平
6,玛多,泊望旅游,负
7,养生队,海南大学校友队,负
8,自在人,荔园路夕阳红,负
9,卓信知产,弘壹邦原点,平


In [19]:
# 创建Team间比赛结果关系
matcher = NodeMatcher(graph)
for i in range(result_df.shape[0]):
    team1 = result_df.iloc[i]['team1']
    team1_node = matcher.match("Team", name=team1).first()
    
    team2 = result_df.iloc[i]['team2']
    team2_node = matcher.match("Team", name=team2).first()
    
    result = result_df.iloc[i]['result']
    
    relationship = Relationship(team1_node, result, team2_node)
    graph.create(relationship)